### Data Prep

In [111]:
import pandas as pd
import re
import numpy as np
from collections import Counter
from nltk.corpus import stopwords

In [112]:
df = pd.read_csv('../datasets/mcdonalds-yelp-negative-reviews.csv')

In [113]:
df.head()

,_unit_id,city,review
0,679455653,Atlanta,"I'm not a huge mcds lover, but I've been to be..."
1,679455654,Atlanta,Terrible customer service. I came in at 9:30pm...
2,679455655,Atlanta,"First they ""lost"" my order, actually they gave..."
3,679455656,Atlanta,I see I'm not the only one giving 1 star. Only...
4,679455657,Atlanta,"Well, it's McDonald's, so you know what the fo..."


In [114]:
df.shape

(1525, 3)

In [115]:
df['city'].value_counts()

Las Vegas      409
Chicago        219
Los Angeles    167
New York       165
Atlanta        130
Houston        105
Portland        97
Dallas          75
Cleveland       71
Name: city, dtype: int64

In [116]:
df['review'] = df['review'].apply(lambda x: x.lower())

In [117]:
review = df['review'].values.tolist()

In [118]:
stopword_list = stopwords.words("english")

def count_words(lines):
    words = Counter() 
    for line in lines:
        line = re.findall("[a-z]+", line) # find character only
        for word in line:
            if word not in stopword_list: # remove stopwords 
                words[word] += 1 
    return words

In [119]:
words = count_words(review)
word_count_df = pd.DataFrame(columns=["word", "frequency"])
word_count_df["word"] = list(words.keys())
word_count_df["frequency"] = list(words.values())
word_count_df.sort_values(by='frequency', ascending=False, inplace=True)

In [120]:
word_count_df

,word,frequency
39,food,887
114,mcdonald,852
15,order,850
7,one,769
12,drive,693
...,...,...
4706,barter,1
4707,protip,1
4708,bottlecaps,1
4709,radaway,1


### Q1: Identify 4 recurring themes/topics that reviewers are unhappy with

Based on top50 frequent word list, we can identify four topics and their corresponding keywords:

1. Drive through (drive, thru)
2. Customer service (service, manager, staff)
3. Wait for too long (minutes)
4. Get the order wrong (wrong)

In [121]:
word_count_df.iloc[:50,:]

,word,frequency
39,food,887
114,mcdonald,852
15,order,850
7,one,769
12,drive,693
11,get,678
50,mcdonalds,580
26,service,540
137,go,534
17,time,533


### Q2: Use regex to group together all occurrences of these phrases

In [122]:
def word_replace(line):
    line = re.sub(r'(drive[ -]thru|drive[ -]through)', '_drive_through_', line)
    line = re.sub(r'(services?|staff|managers?|employees?)', '_service_', line)
    line = re.sub(r'(wait(ing)?s?(ed)?|slow)', '_wait_', line)
    line = re.sub(r'(wrong|forg[eo]ts?)', '_wrong_', line)
    return line

In [123]:
df['review_new'] = df['review'].apply(lambda x:word_replace(x))

### Q3: Perform a word count, both overall, and broken out by city

In [124]:
def count_words(lines):
    words = Counter() 
    for line in lines:
        line = re.findall("[a-z\_]+", line)
        for word in line:
            if word not in stopword_list: # remove stopwords 
                words[word] += 1 
    return words

In [125]:
def word_count_analysis(df, city=None):
    if city:
        df = df[df['city'] == city]
    review_new = df.review_new.values.tolist()
    words = count_words(review_new)
    word_count_df_new = pd.DataFrame(columns=["word", "frequency"])
    word_count_df_new["word"] = list(words.keys())
    word_count_df_new["frequency"] = list(words.values())
    word_count_df_new.sort_values(by='frequency', ascending=False, inplace=True)
    return word_count_df_new

In [126]:
all_city = word_count_analysis(df)
all_city.iloc[:10]

,word,frequency
18,_service_,1211
38,food,887
112,mcdonald,852
15,order,850
7,one,769
11,get,678
12,_drive_through_,590
49,mcdonalds,580
37,_wait_,571
135,go,534


In [127]:
city_list = df['city'].unique().tolist()[:-1]

In [128]:
for city in city_list:
    result = word_count_analysis(df, city)
    print(city)
    print(result.iloc[:10])

Atlanta
                word  frequency
18         _service_        115
15             order         88
38              food         80
112         mcdonald         77
12   _drive_through_         67
7                one         65
49         mcdonalds         59
37            _wait_         58
11               get         56
118         location         45
Las Vegas
                word  frequency
19         _service_        297
83          mcdonald        247
6               food        243
32             order        242
37               one        209
65               get        196
111  _drive_through_        180
69         mcdonalds        149
50                go        139
133           _wait_        138
Dallas
          word  frequency
24   _service_         64
62       order         46
1         food         45
171        one         42
48         get         42
2    mcdonalds         35
51        like         34
254   mcdonald         33
57          go         33
125      wo

### Q4: Provide a few sentences with your findings and business recommendations.

Customer service are mentioned 47% of the time in reviews. Among all cities, Cleveland gets the highest ratio of complaint about service. Comparing to other cities, customers in Cleveland complain more about in-store services(71%) rather than drive-through services(29%). In the review, they often mention how slow the service is so we recommend stores in Cleveland should provide extra traning for front line employees to speed up the service. In addition, slow service and lack of correctness are to blame for the large amount of complaint in drive-through. We suggest them to revise their SOP for delivering drive-through orders. On the other hand, customers in Los Angeles complain more in drive-through service(39%). In the review, customers often stress out with the slow speed of drive-through service. We recommend to install parallel drive through stations. Besides, we observe that in-store service in LA makes more mistakes(18%) in comparison with other cities. We advise that they can set up correctness rate as a new KPI for evaluating overall performance of each store.

In [129]:
topic = ['_drive_through_', '_service_', '_wait_', '_wrong_']
for i in topic:
    df[i] = df['review_new'].apply(lambda x: 1 if x.find(i) != -1 else 0)

In [130]:
all_count = round(df.iloc[:, -4:].mean(),2)
df_all = {i:all_count[i] for i in all_count.index.tolist()}
df_all = pd.DataFrame(data = df_all, index = ['all city'])
df_all

,_drive_through_,_service_,_wait_,_wrong_
all city,0.28,0.47,0.27,0.14


In [131]:
df_by_city = round(df.groupby('city')[topic].mean(),2)
df_compare = pd.concat([df_by_city, df_all])
df_compare

,_drive_through_,_service_,_wait_,_wrong_
Atlanta,0.35,0.55,0.33,0.13
Chicago,0.22,0.47,0.25,0.14
Cleveland,0.28,0.62,0.45,0.15
Dallas,0.32,0.51,0.21,0.17
Houston,0.37,0.52,0.25,0.10
Las Vegas,0.32,0.44,0.27,0.16
Los Angeles,0.39,0.40,0.30,0.18
New York,0.02,0.43,0.24,0.05
Portland,0.27,0.39,0.16,0.12
all city,0.28,0.47,0.27,0.14


In [165]:
df_cleveland = df[(df['city'] == 'Cleveland')]
df_cleveland.groupby('_drive_through_').agg({'_drive_through_': "count", '_wait_': 'mean', '_wrong_':'mean'})

,_drive_through_,_wait_,_wrong_
_drive_through_,,,
0,51,0.333333,0.058824
1,20,0.750000,0.400000


In [166]:
df_la = df[df['city'] == 'Los Angeles']
df_la.groupby('_drive_through_').agg({'_drive_through_': "count", '_wait_': 'mean', '_wrong_':'mean'})

,_drive_through_,_wait_,_wrong_
_drive_through_,,,
0,102,0.196078,0.205882
1,65,0.461538,0.138462


### Explain what some of pitfalls/limitations are of using only a word count analysis to make these inferences. What additional research/steps would you need to do to verify your conclusions?

- We can not distinguish if customers are happy or unhappy with the topics we find if we only use word count analysis. For example, if the customer writes "I like the food in mcdonalds but the service is bad.", we will think the customer is not happy with the food and service. 
- We can not tell how serious the problem is based on how frequently they are metntioned. For example, "The service is bad." and "They provide the worst service in the world" represent different level of anger.
- There are some variants for words such as plural, past tense, etc. However, they will count as different words so that they may not show up in the list of frequent words.
- In Q4, we can observe that there are few negative reviews about drive-through in New York. However, we cannot tell if the reason is that stores in New York provide better drive-through experience. It's possible that stores in New York seldom provide drive-through so that there are few negative reviews. Based on the data, we can not get these information.  

Some possible solutions:

- Apply sentiment analysis in order to know how negative a comment is so that we can pick out those reviews with most negative emotion and conduct word count analysis again.
- Apply stemming as a step to clean up the text to prevent duplicate keywords
- Apply extra dataset(eg. num of stores provide drive-through service in each city) to validate the assumption
